In [1]:
import numpy as np
import itertools
from collections import Counter

In [2]:
FLOOR = 0
EMPTY = 1
OCCUPIED = 2

In [3]:
def read_board(fn):
    board = np.genfromtxt(fn, dtype=np.unicode_, comments=None, delimiter=1)
    int_board = np.zeros(board.shape, dtype=np.ushort)
    height, width = board.shape

    for (i,j) in itertools.product(range(height), range(width)):
        c = board[i,j]

        if c == '.':
            int_board[i, j] = FLOOR
        elif c == 'L':
            int_board[i, j] = EMPTY
        else:
            int_board[i, j] = OCCUPIED
    return int_board

In [4]:
test_board = read_board('./inputs/11_test')

In [5]:
test_board

array([[1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 1, 1, 0, 1, 1]], dtype=uint16)

In [6]:
board = read_board('./inputs/11')

- If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change.

Floor (.) never changes; seats don't move, and nobody sits on the floor.

In [7]:
def in_bounds(i, j, height, width):
     return i in range(height) and j in range(width)

In [8]:
def get_around(i, j, height, width):
    step = [-1, 0, 1]
    
    indices_y = []
    indices_x = []
    
    for (x, y) in itertools.product(step, step):
        # dont visit self
        if x == 0 and y == 0:
            continue
        
        new_i = i+y
        new_j = j+x
        
        if in_bounds(new_i, new_j, height, width):
            indices_y.append(new_i)
            indices_x.append(new_j)

    return indices_y, indices_x

def get_next_board1(board):
    height, width = board.shape
    
    new_board = board.copy()

    for (i,j) in itertools.product(range(height), range(width)):
        state = board[i, j]

        if state != FLOOR:
            around_y, around_x = get_around(i, j, height, width)
            around = board[around_y, around_x]
            count = Counter(around)

            if state == EMPTY and count[OCCUPIED] == 0:
                new_board[i, j] = OCCUPIED
            elif state == OCCUPIED and count[OCCUPIED] >= 4:
                new_board[i, j] = EMPTY

    return new_board

def pt1(board):
    board = board.copy()
    last_board = np.array([])

    for iter in range(200):
        print(iter)
        # print(board)
        if np.array_equal(board, last_board):
            print("Stable.")
            return board
        else:
            last_board = board
            board = get_next_board1(board)

In [9]:
def get_first_in_direction(board, i, j, direction):
    height, width = board.shape
    
    step_x, step_y = direction
    
    for step in range(1, max(height, width)):
        new_i = i+step*step_y
        new_j = j+step*step_x
        
        if in_bounds(new_i, new_j, height, width):
            element = board[new_i, new_j]
            
            if element != FLOOR:
                return element
        else:
            break
        
    return FLOOR

def get_around_extended(board, i, j):
    step = [-1, 0, 1]
    
    place_types = []
    
    # every possible direction
    for (x, y) in itertools.product(step, step):
        # dont visit self
        if x == 0 and y == 0:
            continue
        
        first_in_direction = get_first_in_direction(board, i, j, (x,y))
        place_types.append(first_in_direction)

    return Counter(place_types)

            
def get_next_board2(board):
    height, width = board.shape
    
    new_board = board.copy()

    for (i,j) in itertools.product(range(height), range(width)):
        state = board[i, j]

        if state != FLOOR:
            count = get_around_extended(board, i, j)

            if state == EMPTY and count[OCCUPIED] == 0:
                new_board[i, j] = OCCUPIED
            elif state == OCCUPIED and count[OCCUPIED] >= 5:
                new_board[i, j] = EMPTY

    return new_board
            
def pt2(board):
    board = board.copy()
    last_board = np.array([])

    for iter in range(200):
        print(iter)
        # print(board)
        if np.array_equal(board, last_board):
            print("Stable.")
            return board
        else:
            last_board = board
            board = get_next_board2(board)

In [10]:
final_board_test = pt1(test_board)

0
1
2
3
4
5
6
Stable.


In [11]:
Counter(final_board_test.flatten())

Counter({2: 37, 0: 29, 1: 34})

In [12]:
final_board_test2 = pt2(test_board)

0
1
2
3
4
5
6
7
Stable.


In [13]:
Counter(final_board_test2.flatten())

Counter({2: 26, 0: 29, 1: 45})

In [14]:
board.shape

(98, 96)

In [15]:
final_board = pt1(board)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
Stable.


In [16]:
Counter(final_board.flatten())

Counter({2: 2418, 1: 5146, 0: 1844})

In [19]:
final_board2 = pt2(board)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
Stable.


In [18]:
Counter(final_board2.flatten())

Counter({2: 2144, 1: 5420, 0: 1844})